# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [6]:
# assign the uk_food database to a variable name
establishments = mongo['uk_food']

In [7]:
# review the collections in our database
pprint(establishments.find_one())

TypeError: 'Collection' object is not callable. If you meant to call the 'find_one' method on a 'Database' object it is failing because no such method exists.

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [12]:
#Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {count}")

# Display the first document in the results using pprint
pprint(establishments.find_one(query))

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 67d26364f7b0d69b46966e30, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [ ]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query))  # Convert cursor to list
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
pd.set_option('display.max_columns', None)  # Show all columns
print(df.head(10))

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find establishments with London as the Local Authority and RatingValue >= 4
query = {
    "LocalAuthorityName": {"$regex": "London", "$options": "i"},  # Case-insensitive match for 'London'
    "RatingValue": {"$gte": "4"}  # RatingValue as string, so ensure comparison works correctly
}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments in London with RatingValue >= 4: {count}")

# Display the first document in the results using pprint
pprint(establishments.find_one(query))

In [ ]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query))  # Convert cursor to list
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
pd.set_option('display.max_columns', None)  # Show all columns
print(df.head(10))

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

# Define search parameters
degree_search = 0.01
latitude = 51.5074  # Example latitude (London)
longitude = -0.1278  # Example longitude (London)

# Query: Find establishments within latitude/longitude range and with RatingValue = 5
query = {
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": "5"
}

# Sorting criteria: Sort by hygiene score (ascending)
sort = [("scores.Hygiene", 1)]  # 1 for ascending order

# Limit results (optional)
limit = 10  # Adjust as needed

# Fetch results
results_cursor = establishments.find(query).sort(sort).limit(limit)

# Convert results to a list
results_list = list(results_cursor)

# Print the results
for result in results_list:
    pprint(result)

In [ ]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(results_list)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
# Define the aggregation pipeline
pipeline = [
    {"$match": {"scores.Hygiene": 0}},  # Match hygiene score of 0
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},  # Group by Local Authority
    {"$sort": {"count": -1}}  # Sort in descending order
]

# Execute the aggregation pipeline
results_cursor = establishments.aggregate(pipeline)

# Convert the results to a list
results_list = list(results_cursor)

# Print the number of documents in the result
print(f"Number of Local Authorities with establishments having hygiene score 0: {len(results_list)}")

# Print the first 10 results
pprint(results_list[:10])


In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results_list)

# Rename columns for clarity
df.rename(columns={"_id": "Local Authority", "count": "Establishments with Hygiene Score 0"}, inplace=True)

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
pd.set_option('display.max_columns', None)  # Show all columns
print(df.head(10))